In [5]:
import os
from typing import Literal

import langchain
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from typing_extensions import TypedDict

from buildwithai.langchain_tools import get_summarized_jira_issue, get_ticket_description, set_jira_service, JiraService

print(f"Running with langchain version {langchain.__version__}")

load_dotenv()

chatgptkey = os.getenv("CHATGPT_API_KEY")

s7_jira_user = os.getenv("SIGMA7_JIRA_USER")
s7_jira_token = os.getenv("SIGMA7_JIRA_TOKEN")
s7_jira_url = os.getenv("SIGMA7_JIRA_URL")

jira_user = os.getenv("JIRA_USER")
jira_token = os.getenv("JIRA_API_TOKEN")
jira_url = os.getenv("JIRA_URL")

larry_dawson_jiraservice= JiraService(jira_url, jira_user, jira_token)
sigma7_jiraservice = JiraService(s7_jira_url, s7_jira_user, s7_jira_token)


set_jira_service(larry_dawson_jiraservice)


Running with langchain version 0.3.1


In [6]:
# Define schema for output:
class RouteQuery(TypedDict):
    """Route query to destination expert."""
    destination: Literal["summarizer", "implement"]
    issue_key: str

@tool
def route_to_jira_function(destination: str, issue_key: str) -> str:
    """Route the user's query to either a summarizer or an implementer function."""
    print(destination)
    if destination == "summarizer":
        print("Summarizing JIRA issue")
        return get_summarized_jira_issue(issue_key)
    else:
        print("Getting description of JIRA issue")
        return get_ticket_description(issue_key)

In [7]:

llm = init_chat_model("gpt-4o", model_provider="openai", temperature=0, api_key = chatgptkey)

route_system = "Route the user's query to either a summarizer or description function."
route_prompt = ChatPromptTemplate.from_messages([("system", route_system),
        ("human", "{input}")])

# Instead of writing formatting instructions into the prompt, we
# leverage .with_structured_output to coerce the output into a simple
# schema.
chain = route_prompt | llm.with_structured_output(RouteQuery) | route_to_jira_function

#chain1 = route_prompt | llm.with_structured_output(RouteQuery)
#chain2 = route_to_jira_function
#chain3 = chain1 | chain2 #Yup, this works.


In [8]:
res2 = chain.invoke("summarize JIRA issue FORGE-7")
res2

summarizer
Summarizing JIRA issue


{'summary': 'Agent Design for Automated Systems',
 'description': 'This epic aims to design and develop a robust framework for agents that will automate complex tasks across various domains such as finance, human resources, and management. The design will focus on ensuring high security, role-based access control, and compliance with industry standards. Key areas will include:\n- Developing a base agent class that defines common functionalities and security protocols.\n- Implementing specialized agents such as FinanceAgent, ManagerAgent, and HrAgent with specific capabilities and access controls.\n- Ensuring secure communication protocols between agents and integration points.\n- Establishing a monitoring system for agent activities and anomaly detection.\n- Defining human oversight mechanisms for critical actions.\nThe outcome will be a scalable and secure agent framework that can be adapted to multiple operational needs within the organization.',
 'status': 'To Do',
 'comments': [{'a

In [9]:
res3 = chain.invoke("get description of JIRA issue FORGE-7")
res3

implement
Getting description of JIRA issue


NameError: name 'jira' is not defined

In [44]:
req = f"""who has worked on the ticket with all the detail below:
{res2}
"""

In [45]:
req

"who has worked on the ticket with all the detail below:\n{'summary': 'Agent Design for Automated Systems', 'description': 'This epic aims to design and develop a robust framework for agents that will automate complex tasks across various domains such as finance, human resources, and management. The design will focus on ensuring high security, role-based access control, and compliance with industry standards. Key areas will include:\\n- Developing a base agent class that defines common functionalities and security protocols.\\n- Implementing specialized agents such as FinanceAgent, ManagerAgent, and HrAgent with specific capabilities and access controls.\\n- Ensuring secure communication protocols between agents and integration points.\\n- Establishing a monitoring system for agent activities and anomaly detection.\\n- Defining human oversight mechanisms for critical actions.\\nThe outcome will be a scalable and secure agent framework that can be adapted to multiple operational needs w

In [46]:
who = llm(req)

/var/folders/wc/4xnvpv5s7tjd807525gff95m0000gp/T/ipykernel_56196/398874262.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  who = llm(req)


In [62]:
who.content

'The ticket with the summary "Agent Design for Automated Systems" and the detailed description provided has been commented on by Laurence Dawson. Laurence Dawson mentioned that they are planning to start by implementing a simple agent responsible for reading a request from a Jira ticket for a Python script, implementing the script, and printing instructions on how to run it. The comment was created on 2024-05-06 at 09:18:23.943-0400.'

In [ ]:
#SequentialChain has functionality for chaining multiple functions together. Look at that next.